In [10]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [12]:
!jt -r

Reset css and font defaults in:
/.jupyter/custom &
/Library/Jupyter/nbextensions


In [5]:
import pandas as pd
df_overlap = pd.read_csv('/feature_selection/overlap/Ens-Ref/ens_ref_gene_overlap.csv', index_col=0)

In [6]:
df_overlap.head(2)

,ENS lincRNA gene ID,Ref lncRNA gene ID,Chr region,No. of transcripts in lincrna,No. of transcripts in lncrna,ENS-Ref Overlap ?,same_opp_overlap
0,ENSG00000236648,gene-LOC107984028,1,1,4,Yes,1
1,ENSG00000230714,gene-LOC105372922,1,1,1,Yes,0


In [7]:
from tqdm import tqdm
l_id = []
r_id = []
for i in tqdm(range(len(df_overlap['ENS lincRNA gene ID']))):
    if df_overlap['same_opp_overlap'][i] == 1:
        l_id.append(df_overlap['ENS lincRNA gene ID'][i])
        r_id.append(df_overlap['Ref lncRNA gene ID'][i])

100%|██████████| 6252/6252 [00:00<00:00, 9317.91it/s] 


In [8]:
set_lincrna_id = set(l_id)
set_ref_id = set(r_id)

In [9]:
df_linc_transcript_data = pd.read_csv('/dataset/Ensembl_numeric/all_lincRNA_data/all_lincRNA_sorted_chr/all_lincrna_transcript_data_sorted_chr.csv', index_col=0)

In [10]:
df_linc_transcript_data.head(2)

,SNO,Gene ID,Transcript ID,Biotype,Strand,Seq region Name,Transcript Start,Transcript End,Transcript Length,No. of Exons
0,4916,ENSG00000236423,ENST00000413332,lincRNA,1,1,3900372,3905532,634,3
1,11412,ENSG00000270035,ENST00000602973,lincRNA,-1,1,7698303,7698872,570,1


In [11]:
df_ref_transcript_data = pd.read_csv('/dataset/refseq_numeric/all_type=lncRNA_data/all_lncrna_sorted_chr/all_lncrna_transcript_data_sorted_chr.csv', index_col=0)

In [12]:
df_ref_transcript_data.head(2)

,SNO,Index,RefSeq Gene ID,RefSeq Transcript ID,Dbxref,Parent,gbkey,Type,Seq_id,Strand,Start,End,No. of Exons,Transcript Length
0,1,27,gene-MIR1302-2HG,rna-XR_001737835.1,"GeneID:107985730,Genbank:XR_001737835.1,HGNC:H...",gene-MIR1302-2HG,ncRNA,lnc_RNA,1,1,29926,31295,3,538
1,1508,251564,gene-LOC107985451,rna-XR_001737807.1,"GeneID:107985451,Genbank:XR_001737807.1",gene-LOC107985451,ncRNA,lnc_RNA,1,-1,164909668,164912572,3,603


In [13]:
drop_index = []
for i in tqdm(range(len(df_linc_transcript_data))):
    if df_linc_transcript_data['Gene ID'][i] in set_lincrna_id:
        continue
    else:
        drop_index.append(i)

100%|██████████| 13666/13666 [00:00<00:00, 23614.60it/s]


In [14]:
%%time
df_linc_transcript_data.drop(drop_index, axis=0,inplace=True)

CPU times: user 4.6 ms, sys: 1.78 ms, total: 6.38 ms
Wall time: 15.8 ms


In [15]:
df_linc_transcript_data.reset_index(drop=True, inplace=True)

In [16]:
drop_index = []
for i in tqdm(range(len(df_ref_transcript_data))):
    if df_ref_transcript_data['RefSeq Gene ID'][i] in set_ref_id:
        continue
    else:
        drop_index.append(i)

100%|██████████| 25715/25715 [00:01<00:00, 14234.50it/s]


In [17]:
%%time
df_ref_transcript_data.drop(drop_index, axis=0,inplace=True)

CPU times: user 7.34 ms, sys: 675 µs, total: 8.01 ms
Wall time: 10.2 ms


In [18]:
df_ref_transcript_data.reset_index(drop=True, inplace=True)

In [19]:
df_linc_transcript_data['Seq region Name'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X',
       'Y'], dtype=object)

In [20]:
df_ref_transcript_data['Seq_id'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X',
       'Y'], dtype=object)

In [22]:
len_linc_chr = df_linc_transcript_data['Seq region Name'].value_counts()

In [23]:
len_ref_chr = df_ref_transcript_data['Seq_id'].value_counts()

In [28]:
len_ref_chr['2']

625

In [29]:
lincrna_coords = df_linc_transcript_data[['Transcript Start','Transcript End']].to_numpy()
ref_coords = df_ref_transcript_data[['Start','End']].to_numpy()

In [30]:
a = []
for i in range(0,22,1):
    a.append(str(i+1))

In [31]:
a.append('X')
a.append('Y')
a[0]

'1'

In [36]:
lincrna_gene_id = []
ref_gene_id = []
lincrna_trans_id = []
ref_trans_id = []
no_exons_lincrna = []
no_exons_ref = []
chr_region = []
overlap_len = []

In [37]:
lincrna_id = df_linc_transcript_data['Gene ID']
ref_id = df_ref_transcript_data['RefSeq Gene ID']
l_t_id = df_linc_transcript_data['Transcript ID']
r_t_id = df_ref_transcript_data['RefSeq Transcript ID']
linc_exon_no = df_linc_transcript_data['No. of Exons']
r_exon_no = df_ref_transcript_data['No. of Exons']

In [38]:
def overlap(A, B):
    if A[0] < B[1] and B[0] < A[1]:
        return True
    else:
        return False

In [39]:
def overlap_length(A, B):
    return(min(A[1], B[1]) - max(A[0], B[0]) + 1) 

In [40]:
from tqdm import tqdm
def overlap_transcript():

    for i in tqdm(range(0,24,1)):
        
        if i == 0:
            for j in range(0,len_linc_chr[a[i]]):  
                for k in range(0, len_ref_chr[a[i]]):

                    if overlap(lincrna_coords[j], ref_coords[k]):
                        
                        overlap_len.append(overlap_length(lincrna_coords[j], ref_coords[k]))
                        lincrna_gene_id.append(lincrna_id[j])
                        lincrna_trans_id.append(l_t_id[j])
                        no_exons_lincrna.append(linc_exon_no[j])
                        no_exons_ref.append(r_exon_no[k])
                        ref_gene_id.append(ref_id[k])
                        ref_trans_id.append(r_t_id[k])
                        chr_region.append(a[i])
                            
                    else:
                        
                        continue

        else:
        
            start_index_lincrna = 0
            start_index_ref = 0
            
            for l in range(0,i):
                start_index_ref = start_index_ref + len_ref_chr[a[l]]
                start_index_lincrna = start_index_lincrna + len_linc_chr[a[l]]
            print('start lincrna {0}'.format(start_index_lincrna))
            print('end lincrna {0}'.format(start_index_lincrna + len_linc_chr[a[i]]))
            print('start ref {0}'.format(start_index_ref))
            print('end ref {0}'.format(start_index_ref + len_ref_chr[a[i]]))
            
            
            for j in range(start_index_lincrna, start_index_lincrna + len_linc_chr[a[i]], 1):
                for k in range(start_index_ref, start_index_ref + len_ref_chr[a[i]], 1):
                    
                    if overlap(lincrna_coords[j], ref_coords[k]):
                        
                        overlap_len.append(overlap_length(lincrna_coords[j], ref_coords[k]))
                        lincrna_gene_id.append(lincrna_id[j])
                        lincrna_trans_id.append(l_t_id[j])
                        no_exons_lincrna.append(linc_exon_no[j])
                        no_exons_ref.append(r_exon_no[k])
                        ref_gene_id.append(ref_id[k])
                        ref_trans_id.append(r_t_id[k])
                        chr_region.append(a[i])
                            
                    else:
                        
                        continue

In [41]:
%%time
overlap_transcript()

  4%|▍         | 1/24 [00:01<00:37,  1.65s/it]

start lincrna 760
end lincrna 1730
start ref 599
end ref 1224


  8%|▊         | 2/24 [00:04<00:42,  1.92s/it]

start lincrna 1730
end lincrna 2316
start ref 1224
end ref 1567


 12%|█▎        | 3/24 [00:05<00:34,  1.63s/it]

...

start lincrna 8093
end lincrna 8416
start ref 6758
end ref 6962
start lincrna 8416
end lincrna 8652
start ref 6962
end ref 7177


 88%|████████▊ | 21/24 [00:12<00:00,  5.48it/s]

start lincrna 8652
end lincrna 8922
start ref 7177
end ref 7385
start lincrna 8922
end lincrna 9135
start ref 7385
end ref 7492


100%|██████████| 24/24 [00:13<00:00,  1.84it/s]

start lincrna 9135
end lincrna 9346
start ref 7492
end ref 7626
start lincrna 9346
end lincrna 9424
start ref 7626
end ref 7678
CPU times: user 10.2 s, sys: 234 ms, total: 10.4 s
Wall time: 13.1 s


In [42]:
import csv 
header = ['lincRNA gene ID', 'Ref gene ID', 'lincRNA trans ID', 'Ref trans ID', 'No. of exons lincRNA', 'No. of exons Ref', 'Chr region', 'overlap length']

path = '/feature_selection/overlap/Ens-Ref/trans_overlap.csv'

with open(path, 'wt', newline ='') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(i for i in header)

In [ ]:
# lincrna_gene_id = []
# ref_gene_id = []
# lincrna_trans_id = []
# ref_trans_id = []
# no_exons_lincrna = []
# no_exons_ref = []
# chr_region = []
# overlap_len = []

In [43]:
import pandas as pd

df2 = pd.read_csv(path)

df2[df2.columns[0]] = lincrna_gene_id
df2[df2.columns[1]] = ref_gene_id
df2[df2.columns[2]] = lincrna_trans_id
df2[df2.columns[3]] = ref_trans_id
df2[df2.columns[4]] = no_exons_lincrna
df2[df2.columns[5]] = no_exons_ref
df2[df2.columns[6]] = chr_region
df2[df2.columns[7]] = overlap_len

df2.to_csv(path)

In [49]:
df2.head(2)

,lincRNA gene ID,Ref gene ID,lincRNA trans ID,Ref trans ID,No. of exons lincRNA,No. of exons Ref,Chr region,overlap length
0,ENSG00000236423,gene-LINC01134,ENST00000413332,rna-NR_024455.1,3,4,1,5129
1,ENSG00000270035,gene-LOC105376689,ENST00000602973,rna-XR_002958251.1,1,3,1,570


In [52]:
df_trans_index = df2.set_index(['lincRNA gene ID','Ref gene ID'])

In [54]:
df_trans_index.head(2)

,,lincRNA trans ID,Ref trans ID,No. of exons lincRNA,No. of exons Ref,Chr region,overlap length
lincRNA gene ID,Ref gene ID,,,,,,
ENSG00000236423,gene-LINC01134,ENST00000413332,rna-NR_024455.1,3,4,1,5129
ENSG00000270035,gene-LOC105376689,ENST00000602973,rna-XR_002958251.1,1,3,1,570


In [3]:
(df_trans_index.loc['ENSG00000236423','gene-LINC01134']['overlap length'])

In [86]:
tot_ov_len = []

In [87]:
set_linc_gene_id = set(lincrna_gene_id)
set_ref_gene_id = set(ref_gene_id)

In [88]:
c = 0
for i in tqdm(range(len(l_id))):
    
    if l_id[i] in set_linc_gene_id and r_id[i] in set_ref_gene_id:
        tot_ov_len.append(sum(df_trans_index.loc[l_id[i],r_id[i]]['overlap length']))
        
    else:
        print(i)
        c = c + 1
        tot_ov_len.append('Not Found!')


  0%|          | 0/4557 [00:00<?, ?it/s]/Users/gsoc_ensembl/lib/python3.6/site-packages/ipykernel/__main__.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.

 40%|████      | 1824/4557 [00:08<00:14, 183.59it/s]

1796



100%|██████████| 4557/4557 [00:19<00:00, 234.71it/s]

In [83]:
len(tot_ov_len)

4557

In [89]:
ov_len_for_df = []

In [90]:
count = 0
for i in tqdm(range(len(df_overlap))):
    if df_overlap['same_opp_overlap'][i] == 1:
        ov_len_for_df.append(tot_ov_len[count])
        count = count + 1
    else:
        ov_len_for_df.append('-')


100%|██████████| 6252/6252 [00:00<00:00, 25734.55it/s]

In [93]:
len(ov_len_for_df)

6252

In [94]:
df_overlap['Overlap Length'] = ov_len_for_df

In [95]:
df_overlap.head(2)

,ENS lincRNA gene ID,Ref lncRNA gene ID,Chr region,No. of transcripts in lincrna,No. of transcripts in lncrna,ENS-Ref Overlap ?,same_opp_overlap,Overlap Length
0,ENSG00000236648,gene-LOC107984028,1,1,4,Yes,1,17270
1,ENSG00000230714,gene-LOC105372922,1,1,1,Yes,0,-


In [96]:
df_overlap.to_csv('/feature_selection/overlap/Ens-Ref/ens_ref_gene_overlap_with_ov_len.csv')

In [92]:
print(count)

4557


In [84]:
len(l_id)

4557

In [75]:
len(lincrna_gene_id)

22764

In [76]:
print(c)

1


In [77]:
print(tot_ov_len)

[17270, 1720, 3348, 1710, 15088, 26858, 9807, 14592, 12197, 10303, 23966, 16688, 3428, 1906, 3141, 11622, 7544, 98234, 15462, 353633, 881, 8267, 182905, 89392, 39375, 1623, 382238, 1317, 4180, 9844, 14596, 58710, 300988, 1289, 2509, 2509, 466915, 14209, 1067, 4340, 3222, 7830, 1578, 61912, 7316, 5946, 16060, 20495, 2854, 8277, 190324, 18819, 2091, 1712, 134380, 1323118, 9535, 16764, 13565, 4320, 19094, 6963, 1200, 9533, 110098, 1230, 848, 53464, 8718, 19772, 21057, 1958, 31095, 330, 58251, 726, 28842, 32421, 4276, 437634, 129106, 531, 1903, 1859, 5122, 3825, 563, 4921, 19929, 489790, 293873, 6579, 45356, 5470, 24643, 339, 1167, 14687, 477, 7059, 633, 35205, 32634, 46155, 1662, 17983, 1856, 176769, 59987, 32750, 5624, 4992, 5792, 12967, 3122, 19436, 26980, 2613, 19398, 85793, 16355, 831148, 838937, 94492, 64184, 150312, 16078, 793, 55734, 110310, 37368, 4497, 7336, 612, 35289, 28548, 44604, 31087, 6531, 80016, 321920, 30074, 147480, 33360, 150653, 1683, 18132, 2150, 41635, 8249, 35562, 